In [1]:
'''
Test file for main to add new features:
Features include additional filters by student registration: class name, CRN, DEPT
Will have minimal comments b/c it's a test file
'''
import pandas as pd
import os

### Load session logs csv file and apply first phase of data cleaning on it ###
cwd = os.getcwd()
df = pd.read_csv(cwd + "/(Enc)Data - Coded_SessionLogs.csv")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

df.columns = [col.lower() for col in df.columns]

df = df.rename(columns={'unnamed: 0' : 'id'})
df = df.rename(columns={'"name"' : 'encrypted_name'})
df = df.rename(columns={'services' : 'service'})
df = df.rename(columns={'signintime' : 'sign_in_time'})
df = df.rename(columns={'signouttime' : 'sign_out_time'})
df = df.rename(columns={'period' : 'duration'})

filt = (df['service'] != 'No Sessions')
df = df[filt]

df['sign_in_time'] = pd.to_datetime(df['sign_in_time'])
df['sign_out_time'] = pd.to_datetime(df['sign_out_time'])
df.insert(7, 'hour_in', df['sign_in_time'].apply(lambda x : x.hour))
df.insert(8, 'hour_out', df['sign_out_time'].apply(lambda x : x.hour))
df.insert(9, 'weekday', df['sign_in_time'].apply(lambda x : x.weekday()))

filt = (
    (df['hour_in'] >= 8) & (df['hour_in'] <= 20) &
    (df['hour_out'] >= 8) & (df['hour_out'] <= 20) & (df['weekday'] <= 4)
)
df = df[filt]

df.reset_index(drop=True, inplace=True)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [3]:
df['service'].unique()

array(['In-person Tutoring (STEM Center)', 'Space to Study (STEM Center)',
       'Open Lab (TBA Hours)', 'Virtual Tutoring',
       'Space to Study (MESA)', 'Workshop Attendance', 'Fabrication Lab'],
      dtype=object)

In [4]:
df

,id,encrypted_name,service,course,sign_in_time,sign_out_time,duration,hour_in,hour_out,weekday,tutor
0,1,Shayla,In-person Tutoring (STEM Center),[MATH-252]Calcu/Analytic Geometry II,2022-10-07 09:05:00,2022-10-07 09:16:00,00:11:00,9,9,4,Thiri Wai Wai
1,2,Shayla,In-person Tutoring (STEM Center),[MATH-252]Calcu/Analytic Geometry II,2022-10-05 11:17:00,2022-10-05 11:59:00,00:42:00,11,11,2,Hilary Lin
2,3,Shayla,Space to Study (STEM Center),NaN,2022-09-28 09:03:00,2022-09-28 10:06:00,01:03:00,9,10,2,NaN
3,4,Shayla,In-person Tutoring (STEM Center),[MATH-252]Calcu/Analytic Geometry II,2022-09-21 09:50:00,2022-09-21 10:05:00,00:15:00,9,10,2,Thiri Wai Wai
4,5,Shayla,Space to Study (STEM Center),NaN,2022-09-19 14:11:00,2022-09-19 17:41:00,03:30:00,14,17,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8042,9696,Non-STEM Registered,Space to Study (STEM Center),NaN,2022-08-18 14:58:00,2022-08-18 17:53:00,02:55:00,14,17,3,NaN
8043,9697,Non-STEM Registered,Space to Study (STEM Center),NaN,2022-08-18 13:05:00,2022-08-18 13:48:00,00:43:00,13,13,3,NaN
8044,9698,Non-STEM Registered,Space to Study (STEM Center),NaN,2022-08-17 11:03:00,2022-08-17 12:56:00,01:53:00,11,12,2,NaN
8045,9699,Non-STEM Registered,Space to Study (STEM Center),NaN,2022-08-17 10:27:00,2022-08-17 10:50:00,00:23:00,10,10,2,NaN


In [5]:
# Import registration file and do dirty cleaning
df_reg = pd.read_csv(cwd + "/(Enc)Data - Coded_Registration.csv")
df_reg.columns = [col.lower().replace(' ', '_') for col in df_reg.columns] # rename columns
df_reg['course_dept'] = df_reg['course_dept'].apply(lambda x : x.strip()) # strip white space

# Fix type inconsistencies - convert ints to string
df_reg['crn'] = df_reg['crn'].astype(str)
df_reg['course_num'] = df_reg['course_num'].astype(str)

In [6]:
df_reg

,unnamed:_0,encrypted_name,crn,course_dept,course_num
0,0,Jojo,97649,MATH,200
1,1,Shayla,95003,ENGR,100
2,2,Shayla,97251,IDST,105
3,3,Shayla,96263,MATH,252
4,4,Galileah,97400,ELEC,231
...,...,...,...,...,...
2751,2751,Jeong,80158,BIOL,130
2752,2752,Jeong,91704,CHEM,410
2753,2753,Bansi,95751,CHEM,210
2754,2754,Abdulazeez,80168,BIOL,250


In [7]:
# Merge registration and sessionlog CSV files
course_dept_col = df['course'].str.extract(r'^\[(.+)-[0-9]+\.?[0-9]\]')  # get course_dept w/ regex -> a one col df
course_num_col  = df['course'].str.extract(r'^\[.+-([0-9]+\.?[0-9])\]')  # get course_num w/ regex
df.insert(3, 'course_dept', course_dept_col)
df.insert(4, 'course_num', course_num_col)

In [8]:
df

,id,encrypted_name,service,course_dept,course_num,course,sign_in_time,sign_out_time,duration,hour_in,hour_out,weekday,tutor
0,1,Shayla,In-person Tutoring (STEM Center),MATH,252,[MATH-252]Calcu/Analytic Geometry II,2022-10-07 09:05:00,2022-10-07 09:16:00,00:11:00,9,9,4,Thiri Wai Wai
1,2,Shayla,In-person Tutoring (STEM Center),MATH,252,[MATH-252]Calcu/Analytic Geometry II,2022-10-05 11:17:00,2022-10-05 11:59:00,00:42:00,11,11,2,Hilary Lin
2,3,Shayla,Space to Study (STEM Center),NaN,NaN,NaN,2022-09-28 09:03:00,2022-09-28 10:06:00,01:03:00,9,10,2,NaN
3,4,Shayla,In-person Tutoring (STEM Center),MATH,252,[MATH-252]Calcu/Analytic Geometry II,2022-09-21 09:50:00,2022-09-21 10:05:00,00:15:00,9,10,2,Thiri Wai Wai
4,5,Shayla,Space to Study (STEM Center),NaN,NaN,NaN,2022-09-19 14:11:00,2022-09-19 17:41:00,03:30:00,14,17,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8042,9696,Non-STEM Registered,Space to Study (STEM Center),NaN,NaN,NaN,2022-08-18 14:58:00,2022-08-18 17:53:00,02:55:00,14,17,3,NaN
8043,9697,Non-STEM Registered,Space to Study (STEM Center),NaN,NaN,NaN,2022-08-18 13:05:00,2022-08-18 13:48:00,00:43:00,13,13,3,NaN
8044,9698,Non-STEM Registered,Space to Study (STEM Center),NaN,NaN,NaN,2022-08-17 11:03:00,2022-08-17 12:56:00,01:53:00,11,12,2,NaN
8045,9699,Non-STEM Registered,Space to Study (STEM Center),NaN,NaN,NaN,2022-08-17 10:27:00,2022-08-17 10:50:00,00:23:00,10,10,2,NaN


In [9]:
### We added course_num & course_dept. Now we want to add CRN to this DataFrame - but there's a problem ###
# Make empty Series (list) of CRN - we'll get CRN from registration CSV file, then add here
crn_col = pd.Series(index=range(df.shape[0]), dtype=float)  # empty series

In [10]:
# Test
filt = (
    (df_reg['encrypted_name'] == 'Jojo') &
    (df_reg['course_dept'] == 'MATH')
)
df_reg[filt]['crn']
### PROBEM WITH DATA TYPES HERE BETWEEN SESSIONLOGS DF AND REGISTRATION DF FOR COLUMNS course_name & course_num

0    97649
Name: crn, dtype: object

In [11]:
# Iteratively add CRN to Series to be added to df. Probably .merge() is faster
for idx, row in df.iterrows():
    # Get CRN if there is a course provided for tutoring in the sessionlog
    # If the course is 'NaN', that means that session was not for tutoring, and ignore b/c there's no CRN
    if not pd.isna(df.at[idx, 'course']):
        print(f"Index: {idx}") # Debug test
        # Get session information from session logs
        encrypted_name = row['encrypted_name']  # from sessionlogs df
        course_dept    = row['course_dept']
        course_num     = row['course_num']
        
        # Filter series of registration data that matches the particular session info
        filt = (
            (encrypted_name == df_reg['encrypted_name']) &
            (course_dept    == df_reg['course_dept']) &
            (course_num     == df_reg['course_num'])  # bad code here, need fix later, prob from above comment
        )
        
        # Check for errors
        # Try to get the CRN from registration with the 'filter'
        try:
            crn = df_reg[filt]['crn'].iloc[0]
        except:
            if df_reg[filt].shape[0] != 1:
                # There were no CRNs in registration that matched the session info
                # This is "Tamarsha's Case" - a student that got tutoring for a course but ended up
                # dropping that course. This is because the student is no longer enrolled in the course
                # they got tutoring for, and this is based on the time the registration data was downloaded
                print("PROBLEM: THERE WASN'T ONE MATCH FOR SESSION AND STUDENT'S CRN")
            else:
                # No other case has been detected
                print("There are multiple CRNs that satisfy the criteria")
            # Set CRN to 'Dropped' instead
            crn = 'Dropped'
            # QUESTION: What's the difference btwn'Non-STEM Registered' and Tamarsha's case?
            # Tamarsha's edge case: dropped a class she got tutoring for -> can't get its CRN
            # What is 'Non-STEM Registered' again?
            #
            # 'No Sessions' == Students taking STEM class(es) that haven't been to STEM Center (Dropped all rows)
            # 'Non-STEM Registered' == A student who showed up in our Session Logs,
            #  but did not show up in the registration csv file. ***Why is this?***
            #  Tamarsha's case == Student got tutoring for a class that was later dropped (Set CRN to 'Dropped')
            
            # With this method, 'Non-STEM Registered' sessions will be preserved and included in the df.
            # However, all 'Non-STEM Registered' sessions that were for tutoring will all be marked 'Dropped'
            
            # Another concern: There are 41 sessions where folks got tutoring for a course in the LSKL dept,
            # yet in registration nobody is taking a course in the LSKL dept. Why is this? For some of the
            # less popular courses (not particularly all STEM), students are getting tutoring for them, yet
            # they aren't showing up in the registration. Should we just ignore these b/c they aren't very useful?
            
        print(df_reg[filt])
        print()
        
        crn_col[idx] = crn

Index: 0
   unnamed:_0 encrypted_name    crn course_dept course_num
3           3         Shayla  96263        MATH        252

Index: 1
   unnamed:_0 encrypted_name    crn course_dept course_num
3           3         Shayla  96263        MATH        252

Index: 3
   unnamed:_0 encrypted_name    crn course_dept course_num
3           3         Shayla  96263        MATH        252

Index: 11
   unnamed:_0 encrypted_name    crn course_dept course_num
5           5        Georgiy  92921        BIOL        240

Index: 12
   unnamed:_0 encrypted_name    crn course_dept course_num
5           5        Georgiy  92921        BIOL        240

Index: 13
   unnamed:_0 encrypted_name    crn course_dept course_num
5           5        Georgiy  92921        BIOL        240

Index: 14
   unnamed:_0 encrypted_name    crn course_dept course_num
5           5        Georgiy  92921        BIOL        240

Index: 15
   unnamed:_0 encrypted_name    crn course_dept course_num
5           5        Georgiy  9

     unnamed:_0 encrypted_name    crn course_dept course_num
203         203          Yuren  97357        BIOL        250

Index: 319
     unnamed:_0 encrypted_name    crn course_dept course_num
206         206          Yairy  80166        BIOL        240

Index: 320
     unnamed:_0 encrypted_name    crn course_dept course_num
206         206          Yairy  80166        BIOL        240

Index: 321
     unnamed:_0 encrypted_name    crn course_dept course_num
206         206          Yairy  80166        BIOL        240

Index: 322
     unnamed:_0 encrypted_name    crn course_dept course_num
206         206          Yairy  80166        BIOL        240

Index: 323
     unnamed:_0 encrypted_name    crn course_dept course_num
206         206          Yairy  80166        BIOL        240

Index: 324
     unnamed:_0 encrypted_name    crn course_dept course_num
206         206          Yairy  80166        BIOL        240

Index: 325
     unnamed:_0 encrypted_name    crn course_dept course_num
2

     unnamed:_0 encrypted_name    crn course_dept course_num
315         315         Seumas  92921        BIOL        240

Index: 513
     unnamed:_0 encrypted_name    crn course_dept course_num
315         315         Seumas  92921        BIOL        240

Index: 514
     unnamed:_0 encrypted_name    crn course_dept course_num
315         315         Seumas  92921        BIOL        240

Index: 515
     unnamed:_0 encrypted_name    crn course_dept course_num
315         315         Seumas  92921        BIOL        240

Index: 516
     unnamed:_0 encrypted_name    crn course_dept course_num
321         321          Joren  97641        MATH        800

Index: 517
PROBLEM: THERE WASN'T ONE MATCH FOR SESSION AND STUDENT'S CRN
Empty DataFrame
Columns: [unnamed:_0, encrypted_name, crn, course_dept, course_num]
Index: []

Index: 518
     unnamed:_0 encrypted_name    crn course_dept course_num
320         320          Joren  97640        MATH        200

Index: 532
     unnamed:_0 encrypted_na

     unnamed:_0 encrypted_name    crn course_dept course_num
410         410         Demaya  95797        MATH        251

Index: 730
     unnamed:_0 encrypted_name    crn course_dept course_num
410         410         Demaya  95797        MATH        251

Index: 731
     unnamed:_0 encrypted_name    crn course_dept course_num
408         408         Demaya  97100        COMP        284

Index: 733
     unnamed:_0 encrypted_name    crn course_dept course_num
408         408         Demaya  97100        COMP        284

Index: 736
     unnamed:_0 encrypted_name    crn course_dept course_num
410         410         Demaya  95797        MATH        251

Index: 737
     unnamed:_0 encrypted_name    crn course_dept course_num
410         410         Demaya  95797        MATH        251

Index: 740
     unnamed:_0 encrypted_name    crn course_dept course_num
410         410         Demaya  95797        MATH        251

Index: 747
     unnamed:_0 encrypted_name    crn course_dept course_num
4

PROBLEM: THERE WASN'T ONE MATCH FOR SESSION AND STUDENT'S CRN
Empty DataFrame
Columns: [unnamed:_0, encrypted_name, crn, course_dept, course_num]
Index: []

Index: 1145
PROBLEM: THERE WASN'T ONE MATCH FOR SESSION AND STUDENT'S CRN
Empty DataFrame
Columns: [unnamed:_0, encrypted_name, crn, course_dept, course_num]
Index: []

Index: 1147
     unnamed:_0 encrypted_name    crn course_dept course_num
577         577         Simcha  80699        PHYS        250

Index: 1148
     unnamed:_0 encrypted_name    crn course_dept course_num
577         577         Simcha  80699        PHYS        250

Index: 1155
PROBLEM: THERE WASN'T ONE MATCH FOR SESSION AND STUDENT'S CRN
Empty DataFrame
Columns: [unnamed:_0, encrypted_name, crn, course_dept, course_num]
Index: []

Index: 1166
     unnamed:_0 encrypted_name    crn course_dept course_num
577         577         Simcha  80699        PHYS        250

Index: 1187
     unnamed:_0 encrypted_name    crn course_dept course_num
577         577         Sim

...

     unnamed:_0 encrypted_name    crn course_dept course_num
945         945         Caleia  80165        BIOL        240

Index: 2156
     unnamed:_0 encrypted_name    crn course_dept course_num
945         945         Caleia  80165        BIOL        240

Index: 2157
     unnamed:_0 encrypted_name    crn course_dept course_num
945         945         Caleia  80165        BIOL        240

Index: 2158
     unnamed:_0 encrypted_name    crn course_dept course_num
945         945         Caleia  80165        BIOL        240

Index: 2159
     unnamed:_0 encrypted_name    crn course_dept course_num
945         945         Caleia  80165        BIOL        240

Index: 2160
     unnamed:_0 encrypted_name    crn course_dept course_num
945         945         Caleia  80165        BIOL        240

Index: 2161
     unnamed:_0 encrypted_name    crn course_dept course_num
945         945         Caleia  80165        BIOL        240

Index: 2162
     unnamed:_0 encrypted_name    crn course_dept cours

      unnamed:_0 encrypted_name    crn course_dept course_num
1102        1102    Victorianna  97589        CHEM        210

Index: 2856
      unnamed:_0 encrypted_name    crn course_dept course_num
1102        1102    Victorianna  97589        CHEM        210

Index: 2892
      unnamed:_0 encrypted_name    crn course_dept course_num
1101        1101    Victorianna  96744        BIOL        215

Index: 2904
      unnamed:_0 encrypted_name    crn course_dept course_num
1107        1107      Eustaquio  96273        PHYS        250

Index: 2916
      unnamed:_0 encrypted_name    crn course_dept course_num
1107        1107      Eustaquio  96273        PHYS        250

Index: 2953
      unnamed:_0 encrypted_name    crn course_dept course_num
1107        1107      Eustaquio  96273        PHYS        250

Index: 3000
      unnamed:_0 encrypted_name    crn course_dept course_num
1129        1129        Sanford  97585        COMP        252

Index: 3001
      unnamed:_0 encrypted_name    crn co

      unnamed:_0 encrypted_name    crn course_dept course_num
1390        1390         Enisha  97286        MATH        211

Index: 3887
      unnamed:_0 encrypted_name    crn course_dept course_num
1391        1391         Enisha  96984        MATH        253

Index: 3890
      unnamed:_0 encrypted_name    crn course_dept course_num
1392        1392       Montrice  97583        COMP        250

Index: 3891
      unnamed:_0 encrypted_name    crn course_dept course_num
1392        1392       Montrice  97583        COMP        250

Index: 3892
PROBLEM: THERE WASN'T ONE MATCH FOR SESSION AND STUDENT'S CRN
Empty DataFrame
Columns: [unnamed:_0, encrypted_name, crn, course_dept, course_num]
Index: []

Index: 3893
PROBLEM: THERE WASN'T ONE MATCH FOR SESSION AND STUDENT'S CRN
Empty DataFrame
Columns: [unnamed:_0, encrypted_name, crn, course_dept, course_num]
Index: []

Index: 3894
PROBLEM: THERE WASN'T ONE MATCH FOR SESSION AND STUDENT'S CRN
Empty DataFrame
Columns: [unnamed:_0, encrypted_name

      unnamed:_0 encrypted_name    crn course_dept course_num
1757        1757          Tavey  97594        CHEM        234

Index: 5159
      unnamed:_0 encrypted_name    crn course_dept course_num
1757        1757          Tavey  97594        CHEM        234

Index: 5162
      unnamed:_0 encrypted_name    crn course_dept course_num
1757        1757          Tavey  97594        CHEM        234

Index: 5164
      unnamed:_0 encrypted_name    crn course_dept course_num
1757        1757          Tavey  97594        CHEM        234

Index: 5165
      unnamed:_0 encrypted_name    crn course_dept course_num
1757        1757          Tavey  97594        CHEM        234

Index: 5166
      unnamed:_0 encrypted_name    crn course_dept course_num
1757        1757          Tavey  97594        CHEM        234

Index: 5169
      unnamed:_0 encrypted_name    crn course_dept course_num
1757        1757          Tavey  97594        CHEM        234

Index: 5201
      unnamed:_0 encrypted_name    crn co

      unnamed:_0 encrypted_name    crn course_dept course_num
2663        2663           Znya  92921        BIOL        240

Index: 7467
      unnamed:_0 encrypted_name    crn course_dept course_num
2677        2677         Lizete  96583        BIOL        230

Index: 7468
      unnamed:_0 encrypted_name    crn course_dept course_num
2677        2677         Lizete  96583        BIOL        230

Index: 7469
      unnamed:_0 encrypted_name    crn course_dept course_num
2677        2677         Lizete  96583        BIOL        230

Index: 7470
      unnamed:_0 encrypted_name    crn course_dept course_num
2677        2677         Lizete  96583        BIOL        230

Index: 7471
      unnamed:_0 encrypted_name    crn course_dept course_num
2677        2677         Lizete  96583        BIOL        230

Index: 7472
      unnamed:_0 encrypted_name    crn course_dept course_num
2677        2677         Lizete  96583        BIOL        230

Index: 7473
      unnamed:_0 encrypted_name    crn co

In [12]:
crn_col

0       96263
1       96263
2         NaN
3       96263
4         NaN
        ...  
8042      NaN
8043      NaN
8044      NaN
8045      NaN
8046      NaN
Length: 8047, dtype: object

In [13]:
# Add CRN series to df
df.insert(5, 'crn', crn_col)

In [14]:
df

,id,encrypted_name,service,course_dept,course_num,crn,course,sign_in_time,sign_out_time,duration,hour_in,hour_out,weekday,tutor
0,1,Shayla,In-person Tutoring (STEM Center),MATH,252,96263,[MATH-252]Calcu/Analytic Geometry II,2022-10-07 09:05:00,2022-10-07 09:16:00,00:11:00,9,9,4,Thiri Wai Wai
1,2,Shayla,In-person Tutoring (STEM Center),MATH,252,96263,[MATH-252]Calcu/Analytic Geometry II,2022-10-05 11:17:00,2022-10-05 11:59:00,00:42:00,11,11,2,Hilary Lin
2,3,Shayla,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-09-28 09:03:00,2022-09-28 10:06:00,01:03:00,9,10,2,NaN
3,4,Shayla,In-person Tutoring (STEM Center),MATH,252,96263,[MATH-252]Calcu/Analytic Geometry II,2022-09-21 09:50:00,2022-09-21 10:05:00,00:15:00,9,10,2,Thiri Wai Wai
4,5,Shayla,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-09-19 14:11:00,2022-09-19 17:41:00,03:30:00,14,17,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8042,9696,Non-STEM Registered,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-08-18 14:58:00,2022-08-18 17:53:00,02:55:00,14,17,3,NaN
8043,9697,Non-STEM Registered,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-08-18 13:05:00,2022-08-18 13:48:00,00:43:00,13,13,3,NaN
8044,9698,Non-STEM Registered,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-08-17 11:03:00,2022-08-17 12:56:00,01:53:00,11,12,2,NaN
8045,9699,Non-STEM Registered,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-08-17 10:27:00,2022-08-17 10:50:00,00:23:00,10,10,2,NaN


In [17]:
import sqlite3
conn = sqlite3.connect("sessions.db")
df.to_sql(name="sessions", con=conn, if_exists="replace")

8047

In [18]:
# Problem here !!!
df[((df['service'] == 'Virtual Tutoring') | (df['service'] == 'In-person Tutoring (STEM Center)')) & (df['course_dept'] == 'LSKL')]

,id,encrypted_name,service,course_dept,course_num,crn,course,sign_in_time,sign_out_time,duration,hour_in,hour_out,weekday,tutor
517,725,Joren,In-person Tutoring (STEM Center),LSKL,803,Dropped,[LSKL-803]Supv. Peer Tutoring - PROM,2022-09-19 09:30:00,2022-09-19 10:05:00,00:35:00,9,10,0,Bryan Swartout
1145,1494,Simcha,In-person Tutoring (STEM Center),LSKL,803,Dropped,[LSKL-803]Supv. Peer Tutoring - PROM,2022-11-01 14:46:00,2022-11-01 15:02:00,00:16:00,14,15,1,John Carlo Manuel
1155,1504,Simcha,Virtual Tutoring,LSKL,803,Dropped,[LSKL-803]Supv. Peer Tutoring - PROM,2022-10-27 13:18:00,2022-10-27 13:30:00,00:12:00,13,13,3,John Carlo Manuel
3327,4052,Ludwina,In-person Tutoring (STEM Center),LSKL,803,Dropped,[LSKL-803]Supv. Peer Tutoring - PROM,2022-09-14 14:04:00,2022-09-14 14:15:00,00:11:00,14,14,2,Junior Kyaw
3582,4351,Danzell,In-person Tutoring (STEM Center),LSKL,803,Dropped,[LSKL-803]Supv. Peer Tutoring - PROM,2022-11-09 10:17:00,2022-11-09 10:46:00,00:29:00,10,10,2,Hilary Lin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7908,9561,Non-STEM Registered,In-person Tutoring (STEM Center),LSKL,803,Dropped,[LSKL-803]Supv. Peer Tutoring - PROM,2022-11-07 15:31:00,2022-11-07 16:36:00,01:05:00,15,16,0,Sebastian Vuskovic
7909,9562,Non-STEM Registered,In-person Tutoring (STEM Center),LSKL,803,Dropped,[LSKL-803]Supv. Peer Tutoring - PROM,2022-10-26 15:06:00,2022-10-26 15:33:00,00:27:00,15,15,2,Chuyun Wang
7910,9563,Non-STEM Registered,In-person Tutoring (STEM Center),LSKL,803,Dropped,[LSKL-803]Supv. Peer Tutoring - PROM,2022-10-24 15:00:00,2022-10-24 15:57:00,00:57:00,15,15,0,Lasheana Dilian Husni
8021,9675,Non-STEM Registered,In-person Tutoring (STEM Center),LSKL,803,Dropped,[LSKL-803]Supv. Peer Tutoring - PROM,2022-09-20 16:13:00,2022-09-20 17:54:00,01:41:00,16,17,1,Emma Redalen


In [19]:
# Filter registration for info test
df_reg[(df_reg['course_dept'] == 'COMP') & (df_reg['course_num'] == '250')]

,unnamed:_0,encrypted_name,crn,course_dept,course_num
29,29,Tniyah,97583,COMP,250
188,188,Condoleezza,97583,COMP,250
246,246,Beyda,97584,COMP,250
296,296,Quennell,97583,COMP,250
365,365,Kindall,97584,COMP,250
...,...,...,...,...,...
2485,2485,Amerigo,97583,COMP,250
2551,2551,Latravious,95776,COMP,250
2715,2715,Latea,97583,COMP,250
2725,2725,Jaydehn,97584,COMP,250


In [20]:
df['course'].value_counts()

[BIOL-240]General Microbiology              1949
[BIOL-230]Introduction to Cell Biology       229
[MATH-251]Calculus/Analytic Geometry I       170
[PHYS-250]Physics with Calculus I            114
[CHEM-410]Chem For Health Sciences            99
                                            ... 
[HIST-240]History of Ethnic Groups in CA       1
[CHEM-235]Organic Chemistry II                 1
[CHEM-237]Organic Chemistry Lab I              1
[BIOL-130]Human Biology                        1
[LSKL-800]Supplemental Learn Assis (1)         1
Name: course, Length: 48, dtype: int64

In [21]:
# Check if session logs data was gathered after withdraw deadline (11/15/22)
# Turns out data was gathered on 11/10/22
df[df['sign_in_time'].dt.month == 11].sort_values(by='sign_in_time', ascending=False)

,id,encrypted_name,service,course_dept,course_num,crn,course,sign_in_time,sign_out_time,duration,hour_in,hour_out,weekday,tutor
7840,9488,Non-STEM Registered,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-11-10 19:08:00,2022-11-10 19:41:00,00:33:00,19,19,3,NaN
702,955,Demaya,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-11-10 19:07:00,2022-11-10 19:25:00,00:18:00,19,19,3,NaN
278,397,Coulson,Open Lab (TBA Hours),BIOL,240,84531,[BIOL-240]General Microbiology,2022-11-10 18:20:00,2022-11-10 19:08:00,00:48:00,18,19,3,NaN
110,129,Chess,Open Lab (TBA Hours),BIOL,240,84531,[BIOL-240]General Microbiology,2022-11-10 18:19:00,2022-11-10 19:21:00,01:02:00,18,19,3,NaN
4959,5968,Naly,Open Lab (TBA Hours),BIOL,240,84531,[BIOL-240]General Microbiology,2022-11-10 18:18:00,2022-11-10 19:04:00,00:46:00,18,19,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,1498,Simcha,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-11-01 09:02:00,2022-11-01 09:35:00,00:33:00,9,9,1,NaN
3382,4126,Cheresa,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-11-01 09:02:00,2022-11-01 12:32:00,03:30:00,9,12,1,NaN
2799,3464,Victorianna,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-11-01 08:37:00,2022-11-01 08:57:00,00:20:00,8,8,1,NaN
5722,6906,Jalend,Space to Study (STEM Center),NaN,NaN,NaN,NaN,2022-11-01 08:19:00,2022-11-01 09:06:00,00:47:00,8,9,1,NaN


In [22]:
def make_df_week():
    d = {
        'M'  : [0] * 13,
        'T'  : [0] * 13,
        'W'  : [0] * 13,
        'Th' : [0] * 13,
        'F'  : [0] * 13,
    }
    df = pd.DataFrame(d)
    df.index += 8
    return df

In [23]:
def get_filter(services):
    filt = [False] * df.shape[0]
    for service in services:
        filt = filt | (df['service'] == service)
    return filt

In [24]:
def make_df_with_filter(services):
    filt = get_filter(services)
    filt_df = df[filt]
    filt_df.reset_index(drop=True, inplace=True)
    
    df_week = make_df_week()
    
    for i in range(filt_df.shape[0]):
        hour_in  = filt_df['hour_in'][i]
        hour_out = filt_df['hour_out'][i]
        weekday = filt_df['weekday'][i]
        
        for hour in range(hour_in, hour_out + 1):
            df_week.loc[hour][weekday] += 1
    return df_week